In [3]:
# %% [markdown]
# ### Data Clean up for Yield and Fertiliser
# 
# * Note on assumptions:
#     * averaged over the other variables in the yield_data dataset

# %%
#Imports
import pandas as pd
import openpyxl
import plotly.graph_objects as go
import nbformat


# %%
# Load the data
df = pd.read_csv('InputDataRaw/yield_data.csv')

display((df))

# %%
# Keep only the desired columns
cols_to_keep = ['harvest_year', 'cropping', 'k_amount', 'total_fertilizer_n_amount', 'p_amount', 'grain', 'straw', 'strip']
df = df[cols_to_keep]

# Filter rows where cropping is 'winter wheat'
df = df[df['cropping'] == 'winter wheat']
df = df[df['strip'] == 8]

df = df[((df['harvest_year'] >= 1990) & (df['harvest_year'] <= 2000)) | ((df['harvest_year'] >= 2010) & (df['harvest_year'] <= 2020))]

# Reset index
df = df.reset_index(drop=True)

# Optionally, display the cleaned dataframe
display(df)

# %%
#Add weather data to master dataframe

# Load yearly weather data
weather_df = pd.read_csv('InputDataRaw/weather_data_yearly_cleaned.csv')

# Merge weather data into main df on 'harvest_year'
df = pd.merge(df, weather_df, left_on='harvest_year', right_on='year', how='left')

# Optionally, drop the duplicate 'year' column from weather data
df = df.drop(columns=['year'])

# Display merged dataframe
display(df)

# %%
# Add insect data

# Load yearly insect data for both periods
insect_90s = pd.read_csv('FilteredInput/RISdata_1990_2000_yearly.csv')
insect_10s = pd.read_csv('FilteredInput/RISdata_2010_2020_yearly.csv')

# Combine both periods
insect_df = pd.concat([insect_90s, insect_10s], ignore_index=True)

# Pivot to get one column per species, values are yearly totals
insect_pivot = insect_df.pivot_table(
    index='Year',
    columns='Insect',
    values='Total',
    aggfunc='sum'
)

# Rename columns to match requested format
insect_pivot = insect_pivot.rename(columns=lambda x: f'total_{x}')

# Reset index to make 'Year' a column
insect_pivot = insect_pivot.reset_index()

# Merge with master dataframe on year
df = pd.merge(df, insect_pivot, left_on='harvest_year', right_on='Year', how='left')

# Optionally, drop the duplicate 'Year' column
df = df.drop(columns=['Year'])

# Display updated dataframe
display(df)


# %%
## Add GHG estimates

# IPCC emission factors & conversion values
EF1 = 0.01  # direct N2O-N emission factor
FracGASF, EF4 = 0.1, 0.01
FracLEACH, EF5 = 0.3, 0.0075
GWP_N2O = 273
conv = 44/28

# Upstream emission factors (kg CO2e / kg nutrient)
EF_N, EF_P, EF_K = 5.5, 1.0, 0.6

def calc_emissions(row):
    N = row['total_fertilizer_n_amount']
    P = row['p_amount']
    K = row['k_amount']

    # Direct N2O
    n2o_n_direct = N * EF1
    # Indirect N2O
    n2o_n_vol = N * FracGASF * EF4
    n2o_n_leach = N * FracLEACH * EF5

    n2o_total = (n2o_n_direct + n2o_n_vol + n2o_n_leach) * conv
    co2e_field = n2o_total * GWP_N2O

    # Upstream
    co2e_upstream = (N * EF_N) + (P * EF_P) + (K * EF_K)

    return pd.Series({
        "CO2e_total_kg": co2e_field + co2e_upstream
    })

df = df.join(df.apply(calc_emissions, axis=1))

display(df)

#Source/Justification for upstream EF values (EF_N, EF_P, EF_K):

#CarbonChain reports ~2.6 kg CO₂e/kg for N fertilizer production; P ~1.7 kg CO₂e/kg; K ~0.6 kg CO₂e/kg. carbonchain.com
#Fertilizer Europe & other industrial LCA studies show N fertilizer production emissions can vary between ~3-10 kg CO₂e/kg N depending on energy source and process. fertilizerseurope.com
#IPCC 2006/2019 Guideline refinements provide default emission factors for soil emissions from N application (direct plus indirect). ipcc-nggip.iges.or.jp

# %%
## Export

df.to_excel('FilteredInput/cleaned_yield_data.xlsx', index=False)

# %%
import numpy as np

# Replace '*' with NaN and convert columns to numeric
df['grain'] = pd.to_numeric(df['grain'].replace('*', np.nan))
df['straw'] = pd.to_numeric(df['straw'].replace('*', np.nan))

# Now proceed with your summary and stderr calculations
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
summary_df = df.groupby('harvest_year')[numeric_cols].mean()
stderr_df = df.groupby('harvest_year')[numeric_cols].agg(lambda x: np.std(x, ddof=1) / np.sqrt(len(x)))

# Export both to Excel (each as a separate sheet)
with pd.ExcelWriter('FilteredInput/yield_summary.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Mean')
    stderr_df.to_excel(writer, sheet_name='StdError')

display(summary_df)
display(stderr_df)

# %%
#### KEEP
#This is to show yield of grain and straw over time for the two time periods.
#Description: from this plot, we can see the trends in winter wheat yield of grain and straw over the periods 1990-2000, and 2010-2020. 

import plotly.graph_objects as go

# 1990-2000
summary_90s = summary_df.loc[(summary_df.index >= 1990) & (summary_df.index <= 2000)]
stderr_90s = stderr_df.loc[(stderr_df.index >= 1990) & (stderr_df.index <= 2000)]

fig_90s = go.Figure()

fig_90s.add_trace(go.Scatter(
    x=summary_90s.index,
    y=summary_90s['grain'],
    error_y=dict(type='data', array=stderr_90s['grain']),
    mode='lines+markers',
    name='Grain'
))
fig_90s.add_trace(go.Scatter(
    x=summary_90s.index,
    y=summary_90s['straw'],
    error_y=dict(type='data', array=stderr_90s['straw']),
    mode='lines+markers',
    name='Straw'
))
fig_90s.update_layout(title='Grain and Straw (1990-2000)', xaxis_title='Harvest Year', yaxis_title='Yield (t/ha at 85% dry matter)', legend_title='Variable')
fig_90s.show()

# 2010-2020
summary_10s = summary_df.loc[(summary_df.index >= 2010) & (summary_df.index <= 2020)]
stderr_10s = stderr_df.loc[(stderr_df.index >= 2010) & (stderr_df.index <= 2020)]

fig_10s = go.Figure()

fig_10s.add_trace(go.Scatter(
    x=summary_10s.index,
    y=summary_10s['grain'],
    error_y=dict(type='data', array=stderr_10s['grain']),
    mode='lines+markers',
    name='Grain'
))
fig_10s.add_trace(go.Scatter(
    x=summary_10s.index,
    y=summary_10s['straw'],
    error_y=dict(type='data', array=stderr_10s['straw']),
    mode='lines+markers',
    name='Straw'
))
fig_10s.update_layout(title='Grain and Straw (2010-2020)', xaxis_title='Harvest Year', yaxis_title='Yield (t/ha at 85% dry matter)', legend_title='Variable')
fig_10s.show()

# %%
### KEEP 
#Feriliser use over the two time periods
#Description: You can see that from the first to second time periods, fertiliser use has changed. Phosphorus is no longer applied as existing phosphorus in the soil is sufficient. We discuss the impacts of this on grain yield, insect abundance, and other factors below. 

# Calculate mean fertiliser use for each period
fert_90s = summary_df.loc[(summary_df.index >= 1990) & (summary_df.index <= 2000)][['total_fertilizer_n_amount', 'p_amount', 'k_amount']].mean()
fert_10s = summary_df.loc[(summary_df.index >= 2010) & (summary_df.index <= 2020)][['total_fertilizer_n_amount', 'p_amount', 'k_amount']].mean()

# 1990-2000 Pie Chart
fig_90s_pie = go.Figure(data=[go.Pie(
    labels=['Nitrogen', 'Phosphorus', 'Potassium'],
    values=[fert_90s['total_fertilizer_n_amount'], fert_90s['p_amount'], fert_90s['k_amount']],
    hole=0.3
)])
fig_90s_pie.update_layout(title='Fertiliser Use Split (1990-2000)')
fig_90s_pie.show()

# 2010-2020 Pie Chart
fig_10s_pie = go.Figure(data=[go.Pie(
    labels=['Nitrogen', 'Phosphorus', 'Potassium'],
    values=[fert_10s['total_fertilizer_n_amount'], fert_10s['p_amount'], fert_10s['k_amount']],
    hole=0.3
)])
fig_10s_pie.update_layout(title='Fertiliser Use Split (2010-2020)')
fig_10s_pie.show()





,harvest_year,strip,section,plot,section_1926-67,fertilizer_code,cropping,year_of_wheat,previous_crop,straw_incorporation,...,na_date,mg_factor_level,mg_amount,mg_date,sow_date,harvest_date,cultivar,grain,straw,note
0,1968,21,0,21\0,IA,FYM N2,winter wheat,17.000,winter wheat,no,...,*,nil,0.0,*,1967-12-07,1968-08-26,Cappelle Desprez,4.740,5.280,45
1,1968,22,0,22\0,IA,FYM,winter wheat,17.000,winter wheat,no,...,*,nil,0.0,*,1967-12-07,1968-08-26,Cappelle Desprez,3.930,3.490,46
2,1968,3,0,3\0,IA,Nil,winter wheat,17.000,winter wheat,no,...,*,nil,0.0,*,1967-12-07,1968-08-26,Cappelle Desprez,1.440,0.880,*
3,1968,5,0,5\0,IA,PKNaMg,winter wheat,17.000,winter wheat,no,...,1967-09-19,Mg,11.0,1967-09-19,1967-12-07,1968-08-26,Cappelle Desprez,1.810,1.410,*
4,1968,6,0,6\0,IA,N1PKNaMg,winter wheat,17.000,winter wheat,no,...,1967-09-19,Mg,11.0,1967-09-19,1967-12-07,1968-08-26,Cappelle Desprez,2.640,2.320,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8568,2022,15,9,15\9,VB,N5(P)KMg,winter wheat,64.000,winter wheat,no,...,*,Mg,12.0,2022-04-22,2021-09-22,2022-07-27,Zyatt,7.730,*,*
8569,2022,16,9,16\9,VB,N6(P)KMg,winter wheat,64.000,winter wheat,no,...,*,Mg,12.0,2022-04-22,2021-09-22,2022-07-27,Zyatt,8.940,*,*
8570,2022,17,9,17\9,VB,N1+4+1 PKMg,winter wheat,64.000,winter wheat,no,...,*,Mg,12.0,2022-04-22,2021-09-22,2022-07-27,Zyatt,10.050,*,*
8571,2022,18,9,18\9,VB,N1+2+1 PKMg,winter wheat,64.000,winter wheat,no,...,*,Mg,12.0,2022-04-22,2021-09-22,2022-07-27,Zyatt,8.860,*,*


,harvest_year,cropping,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip
0,1990,winter wheat,90,144,35.0,7.360,*,8
1,1990,winter wheat,90,144,35.0,6.740,2.680,8
2,1990,winter wheat,90,144,35.0,8.100,*,8
3,1990,winter wheat,90,144,35.0,6.440,*,8
4,1990,winter wheat,90,144,35.0,7.720,*,8
...,...,...,...,...,...,...,...,...
153,2019,winter wheat,90,144,0.0,7.685,1.103,8
154,2019,winter wheat,90,144,0.0,4.279,*,8
155,2019,winter wheat,90,144,0.0,8.102,1.585,8
156,2019,winter wheat,90,144,0.0,2.667,2.398,8


,harvest_year,cropping,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip,mean_temp_year,total_rainfall_year
0,1990,winter wheat,90,144,35.0,7.360,*,8,10.471347,709.00356
1,1990,winter wheat,90,144,35.0,6.740,2.680,8,10.471347,709.00356
2,1990,winter wheat,90,144,35.0,8.100,*,8,10.471347,709.00356
3,1990,winter wheat,90,144,35.0,6.440,*,8,10.471347,709.00356
4,1990,winter wheat,90,144,35.0,7.720,*,8,10.471347,709.00356
...,...,...,...,...,...,...,...,...,...,...
153,2019,winter wheat,90,144,0.0,7.685,1.103,8,10.236209,565.20000
154,2019,winter wheat,90,144,0.0,4.279,*,8,10.236209,565.20000
155,2019,winter wheat,90,144,0.0,8.102,1.585,8,10.236209,565.20000
156,2019,winter wheat,90,144,0.0,2.667,2.398,8,10.236209,565.20000


,harvest_year,cropping,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip,mean_temp_year,total_rainfall_year,total_Metopolophium dirhodum,total_Rhopalosiphum padi,total_Sitobion avenae
0,1990,winter wheat,90,144,35.0,7.360,*,8,10.471347,709.00356,579,1256,324
1,1990,winter wheat,90,144,35.0,6.740,2.680,8,10.471347,709.00356,579,1256,324
2,1990,winter wheat,90,144,35.0,8.100,*,8,10.471347,709.00356,579,1256,324
3,1990,winter wheat,90,144,35.0,6.440,*,8,10.471347,709.00356,579,1256,324
4,1990,winter wheat,90,144,35.0,7.720,*,8,10.471347,709.00356,579,1256,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2019,winter wheat,90,144,0.0,7.685,1.103,8,10.236209,565.20000,200,843,145
154,2019,winter wheat,90,144,0.0,4.279,*,8,10.236209,565.20000,200,843,145
155,2019,winter wheat,90,144,0.0,8.102,1.585,8,10.236209,565.20000,200,843,145
156,2019,winter wheat,90,144,0.0,2.667,2.398,8,10.236209,565.20000,200,843,145


,harvest_year,cropping,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip,mean_temp_year,total_rainfall_year,total_Metopolophium dirhodum,total_Rhopalosiphum padi,total_Sitobion avenae,CO2e_total_kg
0,1990,winter wheat,90,144,35.0,7.360,*,8,10.471347,709.00356,579,1256,324,1699.532
1,1990,winter wheat,90,144,35.0,6.740,2.680,8,10.471347,709.00356,579,1256,324,1699.532
2,1990,winter wheat,90,144,35.0,8.100,*,8,10.471347,709.00356,579,1256,324,1699.532
3,1990,winter wheat,90,144,35.0,6.440,*,8,10.471347,709.00356,579,1256,324,1699.532
4,1990,winter wheat,90,144,35.0,7.720,*,8,10.471347,709.00356,579,1256,324,1699.532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2019,winter wheat,90,144,0.0,7.685,1.103,8,10.236209,565.20000,200,843,145,1664.532
154,2019,winter wheat,90,144,0.0,4.279,*,8,10.236209,565.20000,200,843,145,1664.532
155,2019,winter wheat,90,144,0.0,8.102,1.585,8,10.236209,565.20000,200,843,145,1664.532
156,2019,winter wheat,90,144,0.0,2.667,2.398,8,10.236209,565.20000,200,843,145,1664.532


,harvest_year,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip,mean_temp_year,total_rainfall_year,total_Metopolophium dirhodum,total_Rhopalosiphum padi,total_Sitobion avenae,CO2e_total_kg
harvest_year,,,,,,,,,,,,,
1990,1990.0,90.0,144.0,35.0,7.630000,3.275000,8.0,10.471347,709.00356,579.0,1256.0,324.0,1699.532
1991,1991.0,90.0,144.0,35.0,7.207500,6.060000,8.0,9.143046,776.32242,415.0,2683.0,1115.0,1699.532
1992,1992.0,90.0,144.0,35.0,6.883750,4.455000,8.0,10.002850,714.42002,4775.0,4415.0,4911.0,1699.532
1993,1993.0,90.0,144.0,35.0,5.196250,2.540000,8.0,9.488239,935.72110,381.0,1349.0,2123.0,1699.532
1994,1994.0,90.0,144.0,35.0,6.161429,4.190000,8.0,9.593352,934.50516,2583.0,4376.0,6747.0,1699.532
1995,1995.0,90.0,144.0,35.0,5.926250,2.545000,8.0,10.573170,706.90330,229.0,820.0,3983.0,1699.532
1996,1996.0,90.0,144.0,35.0,7.425000,3.827500,8.0,9.243593,652.73870,1101.0,2169.0,12803.0,1699.532
1997,1997.0,90.0,144.0,35.0,6.885000,2.760000,8.0,9.926231,633.17312,5640.0,4720.0,2247.0,1699.532
1998,1998.0,90.0,144.0,35.0,5.166250,4.217500,8.0,10.258893,726.35834,41.0,1374.0,385.0,1699.532


,harvest_year,k_amount,total_fertilizer_n_amount,p_amount,grain,straw,strip,mean_temp_year,total_rainfall_year,total_Metopolophium dirhodum,total_Rhopalosiphum padi,total_Sitobion avenae,CO2e_total_kg
harvest_year,,,,,,,,,,,,,
1990,0.0,0.0,0.0,0.0,0.329832,0.297500,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1991,0.0,0.0,0.0,0.0,0.348029,0.250000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1992,0.0,0.0,0.0,0.0,0.516091,0.512500,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1993,0.0,0.0,0.0,0.0,0.583416,0.280000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1994,0.0,0.0,0.0,0.0,0.489492,0.384856,0.0,0.0,4.641246e-14,0.0,0.0,0.0,9.282491e-14
1995,0.0,0.0,0.0,0.0,0.349785,0.022500,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1996,0.0,0.0,0.0,0.0,0.209122,0.256586,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1997,0.0,0.0,0.0,0.0,0.441814,0.343111,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
1998,0.0,0.0,0.0,0.0,0.580308,0.261954,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00
